In [1]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm_notebook
import os, math, json, time
from yelpapi import YelpAPI

# Credentials and Accessing the API


In [2]:
#Loading in API credentials
with open("/Users/echo/Documents/0424_Data_Enrichment/.secret/yelp_api.json") as f:
    login = json.load(f)
# # Instantiate YelpAPI Variable
yelp_api = YelpAPI(login["api-key"], timeout_s=5.0)

# Define Search

In [7]:
#Setting search definitions
LOCATION = "Austin, TX"
TERM = "dessert"

In [9]:
#Searching yelp api
results = yelp_api.search_query(location=LOCATION, term=TERM)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [17]:
#Getting total results
n_total = results["total"]
n_total

4300

In [15]:
#Getting results per page
n_per_page = len(results["businesses"])
n_per_page

20